In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import random
import os
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

import os
from google.colab import files

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import seaborn as sns
import math
from tqdm.autonotebook import tqdm

In [ ]:
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw




Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
def train(epoch, device, weigth = 1):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var, weigth)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test(device, weigth = 1):
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var, weigth).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

# Task 1: Design the autoencoder structured network for MNIST

In [ ]:
# YOUR CODE!!

class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.z_dim = z_dim
        self.enc_fc1 = nn.Linear(784, h_dim1)
        self.enc_fc2 = nn.Linear(h_dim1, h_dim2)
        self.enc_fc31 = nn.Linear(h_dim2, z_dim)    # mu
        self.enc_fc32 = nn.Linear(h_dim2, z_dim)    # log_var
        
        # sampling z

        # decoder part   
        self.dec_fc1 = nn.Linear(z_dim, h_dim2)
        self.dec_fc2 = nn.Linear(h_dim2, h_dim1)
        self.dec_fc3 = nn.Linear(h_dim1, 784)
        
    def encoder(self, x):
        # return mu, log_var
        out = F.relu(self.enc_fc1(x))
        out = F.relu(self.enc_fc2(out))
        return self.enc_fc31(out), self.enc_fc32(out)
    
    def sampling(self, mu, log_var):
        # return z sample
        # use reparametrization trick!
        sigma = torch.exp(log_var)
        std_z = torch.torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float().to(device)
        return mu + sigma * Variable(std_z, requires_grad=False)
        
    def decoder(self, z):
        # return generated img
        out = F.relu(self.dec_fc1(z))
        out = F.relu(self.dec_fc2(out))
        return torch.sigmoid(self.dec_fc3(out))
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=50)
if torch.cuda.is_available():
    vae.to(device)

# Task 2: Design the loss function for autoencoder with weight of KLD term

In [ ]:
optimizer = optim.Adam(vae.parameters())
def loss_function(recon_x, x, mu, log_var, weight = 1):
    # return reconstruction error + weight * KL divergence losses
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = weight * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE - KLD

In [ ]:
from tqdm.autonotebook import tqdm

for epoch in tqdm(range(1, 51)):
    train(epoch, device)
    test(device)

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.954285
Train Epoch: 1 [12800/60000 (21%)]	Loss: 200.813065
Train Epoch: 1 [25600/60000 (43%)]	Loss: 193.499557
Train Epoch: 1 [38400/60000 (64%)]	Loss: 169.322037
Train Epoch: 1 [51200/60000 (85%)]	Loss: 159.723419
====> Epoch: 1 Average loss: 188.2943
====> Test set loss: 149.8646
Train Epoch: 2 [0/60000 (0%)]	Loss: 154.290482
Train Epoch: 2 [12800/60000 (21%)]	Loss: 141.943680
Train Epoch: 2 [25600/60000 (43%)]	Loss: 132.770340
Train Epoch: 2 [38400/60000 (64%)]	Loss: 140.850296
Train Epoch: 2 [51200/60000 (85%)]	Loss: 134.204330
====> Epoch: 2 Average loss: 136.9308
====> Test set loss: 127.3043
Train Epoch: 3 [0/60000 (0%)]	Loss: 122.794670
Train Epoch: 3 [12800/60000 (21%)]	Loss: 126.146461
Train Epoch: 3 [25600/60000 (43%)]	Loss: 124.235184
Train Epoch: 3 [38400/60000 (64%)]	Loss: 122.999245
Train Epoch: 3 [51200/60000 (85%)]	Loss: 120.831833
====> Epoch: 3 Average loss: 123.9106
====> Test set loss: 119.3393
Train Epoch: 4 [0/60000 (0%)]	L

# Task 3

In [ ]:
with torch.no_grad():
    # random sampled latent points
    z_dim = 50
    z = torch.randn(64, z_dim)     
    z = torch.tensor(z).to(device)
    sample = vae.decoder(z.float())
    if not os.path.exists('./samples'):
        os.makedirs('./samples')
    save_image(sample.view(64, 1, 28, 28), './samples/sample' + '.png')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
!ls -la ./samples/

total 308
drwxr-xr-x 2 root root  4096 Jun 22 15:54 .
drwxr-xr-x 1 root root  4096 Jun 22 15:19 ..
-rw-r--r-- 1 root root 61740 Jun 22 15:36 sample_10.png
-rw-r--r-- 1 root root 63600 Jun 22 15:45 sample_25.png
-rw-r--r-- 1 root root 63451 Jun 22 15:28 sample_2.png
-rw-r--r-- 1 root root 62783 Jun 22 15:54 sample_50.png
-rw-r--r-- 1 root root 40983 Jun 22 15:19 sample.png


In [ ]:
# files.download('./samples/sample.png')

# Homework 4

### Functions

In [ ]:
def train_vae(vae, weigth=1):
    optimizer = optim.Adam(vae.parameters())
    vae.train()
    for epoch in tqdm(range(1, 51)):
        train_epoch(vae, epoch, device, optimizer, weigth=weigth)
        test_epoch(vae, device, weigth=weigth)

In [ ]:
def random_samples_image(vae, n_samples, z_dim, img_name):
    with torch.no_grad():
        vae.eval()
        z = torch.randn(n_samples, z_dim)
        z = torch.tensor(z).to(device)
        samples = vae.decoder(z.float())
        if not os.path.exists('./samples'):
            os.makedirs('./samples')
        print("Generating image...")
        save_image(samples.view(n_samples, 1, 28, 28), './samples/' + img_name + '.png', nrow = int(math.sqrt(n_samples)))

In [21]:
def grid_samples_image(vae, n_samples, img_name):
    """
    z_dim must be equal to 2
    """
    lat = int(math.sqrt(n_samples))
    with torch.no_grad():
        vae.eval()
        span = torch.linspace(start = -4, end = 4, steps = lat)
        elems = []
        for a in range(lat):
            for b in range(lat):
                elems.append(torch.cat([span[a].unsqueeze(dim=0).unsqueeze(dim=1), span[b].unsqueeze(dim=0).unsqueeze(dim=1)], dim = 1))
        z1 = torch.cat([elem for elem in elems], dim = 0)
        z = torch.tensor(z1).to(device)
        samples = vae.decoder(z.float())
        if not os.path.exists('./samples'):
            os.makedirs('./samples')
        print("Generating image...")
        save_image(samples.view(n_samples, 1, 28, 28), './samples/' + img_name + '.png', nrow = int(math.sqrt(n_samples)))

In [ ]:
def show_samples(images, row, col, image_shape, name="Unknown", save=False, shift=False):
    sns.axes_style("white")
    num_images = row*col
    if shift:
        images = (images+1.)/2.
    fig = plt.figure(figsize=(col, row))
    grid = ImageGrid(fig, 111,
                     nrows_ncols=(row, col),
                     axes_pad=0.)
    for i in range(num_images):
        im = images[i].reshape(image_shape)
        axis = grid[i]
        axis.axis('off')
        axis.imshow(im)
    plt.axis('off')
    plt.tight_layout()
    if save:
        fig.savefig('./samples/'+name+'.png', bbox_inches="tight", pad_inches=0, format='png')
    else:
        plt.show()

In [ ]:
def train_epoch(vae, epoch, device, optimizer, weigth = 1):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var, weigth)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test_epoch(vae, device, weigth = 1):
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var, weigth).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

## Problem2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
configurations = [2, 10, 25, 50]

for conf in configurations:
    vae_c = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=conf)
    if torch.cuda.is_available():
        vae_c.to(device)
    train_vae(vae_c)
    random_samples_image(vae_c, 100, conf, 'sample_'+str(conf))
    # files.download('./samples/' + 'sample_' + str(conf) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.925293
Train Epoch: 1 [12800/60000 (21%)]	Loss: 196.347992
Train Epoch: 1 [25600/60000 (43%)]	Loss: 186.169250
Train Epoch: 1 [38400/60000 (64%)]	Loss: 170.765884
Train Epoch: 1 [51200/60000 (85%)]	Loss: 169.386475
====> Epoch: 1 Average loss: 188.9437
====> Test set loss: 167.2820
Train Epoch: 2 [0/60000 (0%)]	Loss: 169.525742
Train Epoch: 2 [12800/60000 (21%)]	Loss: 160.616516
Train Epoch: 2 [25600/60000 (43%)]	Loss: 163.996796
Train Epoch: 2 [38400/60000 (64%)]	Loss: 157.828278
Train Epoch: 2 [51200/60000 (85%)]	Loss: 159.814606
====> Epoch: 2 Average loss: 162.0612
====> Test set loss: 158.2659
Train Epoch: 3 [0/60000 (0%)]	Loss: 158.784180
Train Epoch: 3 [12800/60000 (21%)]	Loss: 165.724426
Train Epoch: 3 [25600/60000 (43%)]	Loss: 161.749390
Train Epoch: 3 [38400/60000 (64%)]	Loss: 148.501755
Train Epoch: 3 [51200/60000 (85%)]	Loss: 165.771957
====> Epoch: 3 Average loss: 155.5265
====> Test set loss: 154.1741
Train Epoch: 4 [0/60000 (0%)]	L

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Train Epoch: 1 [0/60000 (0%)]	Loss: 543.429382
Train Epoch: 1 [12800/60000 (21%)]	Loss: 196.285995
Train Epoch: 1 [25600/60000 (43%)]	Loss: 180.768784
Train Epoch: 1 [38400/60000 (64%)]	Loss: 154.714600
Train Epoch: 1 [51200/60000 (85%)]	Loss: 148.685684
====> Epoch: 1 Average loss: 182.6478
====> Test set loss: 147.9590
Train Epoch: 2 [0/60000 (0%)]	Loss: 145.457901
Train Epoch: 2 [12800/60000 (21%)]	Loss: 137.567307
Train Epoch: 2 [25600/60000 (43%)]	Loss: 138.094849
Train Epoch: 2 [38400/60000 (64%)]	Loss: 135.156296
Train Epoch: 2 [51200/60000 (85%)]	Loss: 135.996628
====> Epoch: 2 Average loss: 139.5121
====> Test set loss: 133.1990
Train Epoch: 3 [0/60000 (0%)]	Loss: 125.673241
Train Epoch: 3 [12800/60000 (21%)]	Loss: 127.966476
Train Epoch: 3 [25600/60000 (43%)]	Loss: 132.094757
Train Epoch: 3 [38400/60000 (64%)]	Loss: 135.109467
Train Epoch: 3 [51200/60000 (85%)]	Loss: 126.198387
====> Epoch: 3 Average loss: 130.7646
====> Test set loss: 126.7415
Train Epoch: 4 [0/60000 (0%)]	L

Train Epoch: 1 [0/60000 (0%)]	Loss: 545.882080
Train Epoch: 1 [12800/60000 (21%)]	Loss: 196.978897
Train Epoch: 1 [25600/60000 (43%)]	Loss: 179.994095
Train Epoch: 1 [38400/60000 (64%)]	Loss: 170.248993
Train Epoch: 1 [51200/60000 (85%)]	Loss: 150.595856
====> Epoch: 1 Average loss: 182.5759
====> Test set loss: 141.9503
Train Epoch: 2 [0/60000 (0%)]	Loss: 146.281387
Train Epoch: 2 [12800/60000 (21%)]	Loss: 139.053680
Train Epoch: 2 [25600/60000 (43%)]	Loss: 129.387451
Train Epoch: 2 [38400/60000 (64%)]	Loss: 120.029564
Train Epoch: 2 [51200/60000 (85%)]	Loss: 124.199860
====> Epoch: 2 Average loss: 131.6153
====> Test set loss: 123.1641
Train Epoch: 3 [0/60000 (0%)]	Loss: 127.445312
Train Epoch: 3 [12800/60000 (21%)]	Loss: 123.705902
Train Epoch: 3 [25600/60000 (43%)]	Loss: 123.612946
Train Epoch: 3 [38400/60000 (64%)]	Loss: 120.937126
Train Epoch: 3 [51200/60000 (85%)]	Loss: 126.216782
====> Epoch: 3 Average loss: 121.1146
====> Test set loss: 117.6153
Train Epoch: 4 [0/60000 (0%)]	L

Train Epoch: 1 [0/60000 (0%)]	Loss: 545.161438
Train Epoch: 1 [12800/60000 (21%)]	Loss: 203.209274
Train Epoch: 1 [25600/60000 (43%)]	Loss: 185.206879
Train Epoch: 1 [38400/60000 (64%)]	Loss: 173.990341
Train Epoch: 1 [51200/60000 (85%)]	Loss: 169.882050
====> Epoch: 1 Average loss: 190.5092
====> Test set loss: 155.3668
Train Epoch: 2 [0/60000 (0%)]	Loss: 156.822464
Train Epoch: 2 [12800/60000 (21%)]	Loss: 145.234695
Train Epoch: 2 [25600/60000 (43%)]	Loss: 144.379333
Train Epoch: 2 [38400/60000 (64%)]	Loss: 133.728195
Train Epoch: 2 [51200/60000 (85%)]	Loss: 130.477417
====> Epoch: 2 Average loss: 139.1445
====> Test set loss: 128.3710
Train Epoch: 3 [0/60000 (0%)]	Loss: 126.851830
Train Epoch: 3 [12800/60000 (21%)]	Loss: 127.288513
Train Epoch: 3 [25600/60000 (43%)]	Loss: 125.174965
Train Epoch: 3 [38400/60000 (64%)]	Loss: 121.497894
Train Epoch: 3 [51200/60000 (85%)]	Loss: 120.764450
====> Epoch: 3 Average loss: 125.3655
====> Test set loss: 121.7343
Train Epoch: 4 [0/60000 (0%)]	L

In [ ]:
for conf in configurations:
    files.download('./samples/' + 'sample_' + str(conf) + '.png')

## Problem3

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
configurations = [1, 5, 10, 40]
for conf in configurations:
    vae_c = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2)
    if torch.cuda.is_available():
        vae_c.to(device)
    train_vae(vae_c, weigth=conf)
    grid_samples_image(vae_c, 100, 'sample_prob3_'+str(conf))
    # files.download('./samples/' + 'sample_prob3_' + str(conf) + '.png')


Train Epoch: 1 [0/60000 (0%)]	Loss: 545.775696
Train Epoch: 1 [12800/60000 (21%)]	Loss: 200.532684
Train Epoch: 1 [25600/60000 (43%)]	Loss: 180.806747
Train Epoch: 1 [38400/60000 (64%)]	Loss: 179.802628
Train Epoch: 1 [51200/60000 (85%)]	Loss: 170.553955
====> Epoch: 1 Average loss: 190.6835
====> Test set loss: 167.1185
Train Epoch: 2 [0/60000 (0%)]	Loss: 165.212326
Train Epoch: 2 [12800/60000 (21%)]	Loss: 158.549393
Train Epoch: 2 [25600/60000 (43%)]	Loss: 158.357819
Train Epoch: 2 [38400/60000 (64%)]	Loss: 159.524689
Train Epoch: 2 [51200/60000 (85%)]	Loss: 155.863922
====> Epoch: 2 Average loss: 161.3808
====> Test set loss: 157.3404
Train Epoch: 3 [0/60000 (0%)]	Loss: 158.146652
Train Epoch: 3 [12800/60000 (21%)]	Loss: 164.576843
Train Epoch: 3 [25600/60000 (43%)]	Loss: 153.164032
Train Epoch: 3 [38400/60000 (64%)]	Loss: 150.198944
Train Epoch: 3 [51200/60000 (85%)]	Loss: 152.345337
====> Epoch: 3 Average loss: 155.2925
====> Test set loss: 153.3622
Train Epoch: 4 [0/60000 (0%)]	L

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Train Epoch: 1 [0/60000 (0%)]	Loss: 544.880798
Train Epoch: 1 [12800/60000 (21%)]	Loss: 196.338821
Train Epoch: 1 [25600/60000 (43%)]	Loss: 195.959000
Train Epoch: 1 [38400/60000 (64%)]	Loss: 202.460754
Train Epoch: 1 [51200/60000 (85%)]	Loss: 197.003448
====> Epoch: 1 Average loss: 205.0168
====> Test set loss: 197.1689
Train Epoch: 2 [0/60000 (0%)]	Loss: 196.803238
Train Epoch: 2 [12800/60000 (21%)]	Loss: 194.274368
Train Epoch: 2 [25600/60000 (43%)]	Loss: 198.677612
Train Epoch: 2 [38400/60000 (64%)]	Loss: 191.755051
Train Epoch: 2 [51200/60000 (85%)]	Loss: 187.821075
====> Epoch: 2 Average loss: 193.1473
====> Test set loss: 185.7422
Train Epoch: 3 [0/60000 (0%)]	Loss: 182.418182
Train Epoch: 3 [12800/60000 (21%)]	Loss: 179.809631
Train Epoch: 3 [25600/60000 (43%)]	Loss: 175.598236
Train Epoch: 3 [38400/60000 (64%)]	Loss: 177.997528
Train Epoch: 3 [51200/60000 (85%)]	Loss: 183.185196
====> Epoch: 3 Average loss: 180.5852
====> Test set loss: 176.5383
Train Epoch: 4 [0/60000 (0%)]	L

Train Epoch: 1 [0/60000 (0%)]	Loss: 545.653076
Train Epoch: 1 [12800/60000 (21%)]	Loss: 205.822891
Train Epoch: 1 [25600/60000 (43%)]	Loss: 199.833511
Train Epoch: 1 [38400/60000 (64%)]	Loss: 196.905243
Train Epoch: 1 [51200/60000 (85%)]	Loss: 202.566269
====> Epoch: 1 Average loss: 207.5477
====> Test set loss: 200.3913
Train Epoch: 2 [0/60000 (0%)]	Loss: 194.125427
Train Epoch: 2 [12800/60000 (21%)]	Loss: 205.545547
Train Epoch: 2 [25600/60000 (43%)]	Loss: 197.585632
Train Epoch: 2 [38400/60000 (64%)]	Loss: 199.889542
Train Epoch: 2 [51200/60000 (85%)]	Loss: 198.311996
====> Epoch: 2 Average loss: 200.1381
====> Test set loss: 198.8195
Train Epoch: 3 [0/60000 (0%)]	Loss: 199.703537
Train Epoch: 3 [12800/60000 (21%)]	Loss: 204.806793
Train Epoch: 3 [25600/60000 (43%)]	Loss: 194.929398
Train Epoch: 3 [38400/60000 (64%)]	Loss: 195.299667
Train Epoch: 3 [51200/60000 (85%)]	Loss: 198.344147
====> Epoch: 3 Average loss: 198.8852
====> Test set loss: 197.8037
Train Epoch: 4 [0/60000 (0%)]	L

Train Epoch: 1 [0/60000 (0%)]	Loss: 546.289124
Train Epoch: 1 [12800/60000 (21%)]	Loss: 208.703857
Train Epoch: 1 [25600/60000 (43%)]	Loss: 202.054474
Train Epoch: 1 [38400/60000 (64%)]	Loss: 204.802719
Train Epoch: 1 [51200/60000 (85%)]	Loss: 207.723267
====> Epoch: 1 Average loss: 212.0386
====> Test set loss: 206.1523
Train Epoch: 2 [0/60000 (0%)]	Loss: 202.587128
Train Epoch: 2 [12800/60000 (21%)]	Loss: 206.294052
Train Epoch: 2 [25600/60000 (43%)]	Loss: 206.547546
Train Epoch: 2 [38400/60000 (64%)]	Loss: 201.607925
Train Epoch: 2 [51200/60000 (85%)]	Loss: 204.578674
====> Epoch: 2 Average loss: 206.2594
====> Test set loss: 206.0388
Train Epoch: 3 [0/60000 (0%)]	Loss: 198.550888
Train Epoch: 3 [12800/60000 (21%)]	Loss: 212.164963
Train Epoch: 3 [25600/60000 (43%)]	Loss: 208.532104
Train Epoch: 3 [38400/60000 (64%)]	Loss: 211.518082
Train Epoch: 3 [51200/60000 (85%)]	Loss: 208.221130
====> Epoch: 3 Average loss: 206.1101
====> Test set loss: 205.7875
Train Epoch: 4 [0/60000 (0%)]	L

In [24]:
for conf in configurations:
    files.download('./samples/' + 'sample_prob3_' + str(conf) + '.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>